#1. Instalation and import packages

In [1]:
#THIS IS FOR USING JUST ONE GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
# Install the evaluate library for evaluation metrics like accuracy
!pip install evaluate

In [3]:
!pip install datasets transformers


In [4]:
!pip install --upgrade transformers

In [5]:
!pip install datasets

In [6]:
import re
import json
import torch
from tqdm import tqdm
from datasets import load_dataset
from transformers import (
    GPT2TokenizerFast,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import evaluate

/home/maria/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Loading and Splitting the Dataset
 - We load the gsm8k dataset which contains grade-school math word problems:

- The official training split is splitt into 80% training and 10% validation sets.

- The provided test set is kept as is for final evaluation.

In [7]:
# 1. Load & split GSM8K
gsm8k     = load_dataset("gsm8k", "main")
train_val = gsm8k["train"].shuffle(seed=42)
test_data = gsm8k["test"]                   # Official held-out test split

# 80/10 split of train_val → train / val
val_size      = int(0.1 * len(train_val))
split         = train_val.train_test_split(test_size=val_size, seed=42)
train_data    = split["train"]
val_data      = split["test"]

print(f" Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

 Train: 6726, Val: 747, Test: 1319


# 3. Loading the Model and Tokenizer
We load the GPT-2 model and tokenizer
The tokenizer padding token is set to the end of sequence token (as GPT-2 was not trained with padding).
The model is resized to match the tokenizer’s vocabulary and moved to GPU if available.

In [8]:
# 2. Tokenizer & Model
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Preprocessing with prompt-masking
max_length = 256

# 4. Preprocessing the Dataset
We define a function preprocess that:

* Converts each question answer pair into a prompt format: "Q: question\nA: answer"

* Applies tokenization with padding and truncation

* Masks the prompt portion of the input so that the model is only trained to predict the answer

* Applies this preprocessing to both the training and validation sets

Masks the prompt so the model only learns to predict the answer, not the question

In [9]:
def preprocess_fn(examples):
    input_ids, attention_mask, labels = [], [], []
    for q, a in zip(examples["question"], examples["answer"]):
        prompt = f"Q: {q.strip()}\nA:"
        full   = prompt + " " + a.strip() + "\n"

        tok_full = tokenizer(
            full,
            truncation=True,
            padding="max_length",
            max_length=max_length,
        )
        # mask the prompt tokens
        prompt_len = len(tokenizer(prompt)["input_ids"])
        label = [-100] * prompt_len + tok_full["input_ids"][prompt_len:]
        # pad the rest of the labels to max_length
        label = label + [-100] * (max_length - len(label))

        input_ids.append(tok_full["input_ids"])
        attention_mask.append(tok_full["attention_mask"])
        labels.append(label)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

tokenized_train = train_data.map(
    preprocess_fn,
    batched=True,
    remove_columns=train_data.column_names,
)
tokenized_val = val_data.map(
    preprocess_fn,
    batched=True,
    remove_columns=val_data.column_names,
)

Map: 100%|██████████| 747/747 [00:00<00:00, 2083.87 examples/s]


In [10]:
# 4. Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


# 5. Training the Model
* We adjusted  Hyperparameters
* Evaluation and checkpoint saving frequency
* trainer.train() to start fine tuning the model and finally we evaluate the model

In [11]:
# 5. TrainingArguments & Trainer
# how often (in steps) to run eval & checkpoint
# here: once every 1 000 optimization steps
training_args = TrainingArguments(
    output_dir="./gpt2-gsm8k-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    logging_steps=100,
    save_steps=1000,
    eval_steps=1000,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to=[],            #  turn off W&B (and all other loggers)
    run_name="gsm8k-finetune" #  optional, avoids duplicate-name warnings
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    # tokenizer=tokenizer    you can drop this now to silence the deprecation warning
)


# 6. Fine-tune
print(" Starting fine-tuning…")
trainer.train()
trainer.save_model("./gpt2-gsm8k-finetuned")

# 7. Load fine-tuned model for generation
model = GPT2LMHeadModel.from_pretrained("./gpt2-gsm8k-finetuned")
model.to(device)
model.eval()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


 Starting fine-tuning…


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.860900
200,2.147900
300,1.927800
400,1.848900
500,1.798200
600,1.764200
700,1.719000
800,1.680200
900,1.611700
1000,1.578800


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# 6. Numeric Answer Evaluation Without Reasoning (Zero-Shot)
Generates answers for the test set and computes baseline numeric accuracy by comparing predicted and true final numbers

In [12]:
import re
import pandas as pd
from tqdm.auto import tqdm

def extract_number(s):
    # grab the *last* integer or decimal in the string
    nums = re.findall(r"-?\d+\.?\d*", s)
    return nums[-1] if nums else None

results = []
for ex in tqdm(test_data, desc="Gen test set"):
    q, gold = ex["question"], ex["answer"].strip()
    prompt = f"Q: {q}\nA:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to(device)

    out = model.generate(
        **inputs,
        max_new_tokens=64,
        num_beams=1,
        pad_token_id=tokenizer.pad_token_id
    )

    gen_tokens = out[0][ inputs["input_ids"].shape[-1]: ]
    pred = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

    gold_num = extract_number(gold)
    pred_num = extract_number(pred)
    correct  = (gold_num == pred_num)

    results.append({
        "question": q,
        "gold_full":    gold,
        "pred_full":    pred,
        "gold_number":  gold_num,
        "pred_number":  pred_num,
        "correct":      correct
    })

baseline_df  = pd.DataFrame(results)
baseline_acc = baseline_df["correct"].mean()
print(f" Baseline numeric-only accuracy: {baseline_acc:.3%}")


Gen test set: 100%|██████████| 1319/1319 [08:42<00:00,  2.53it/s]

 Baseline numeric-only accuracy: 2.123%


# 7. Save Evaluation Results to JSON

In [13]:
import json

# all_results = your list of dicts from the loop above
with open("all_predictions.json", "w") as f:
    json.dump(results, f, indent=2)

perfect = [r for r in results if r["correct"]]
with open("perfect_subset.json", "w") as f:
    json.dump(perfect, f, indent=2)

print(f"Saved {len(results)} total preds and {len(perfect)} perfect ones.")


Saved 1319 total preds and 28 perfect ones.


Let's try with zero shot!

# 8. Extract Final Answer from Gold Labels

In [14]:
import re
from tqdm.auto import tqdm

def extract_number(s):
    # grab the *last* integer or decimal in the string
    nums = re.findall(r"-?\d+\.?\d*", s)
    return nums[-1] if nums else None

cot_results = []
for i, ex in enumerate(tqdm(test_data, desc="CoT test set")):
    q, raw_gold = ex["question"], ex["answer"]
    gold_num = extract_number(raw_gold)

    prompt = f"Q: {q}\nA: Let’s think step by step."
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to(device)

    out = model.generate(
        **inputs,
        max_new_tokens=128,
        num_beams=1,
        pad_token_id=tokenizer.pad_token_id
    )

    text = tokenizer.decode(out[0], skip_special_tokens=True)

    # DEBUG: print first few to see the format
    if i < 5:
        print("––––––––––––––––––")
        print(text)
        print("gold_num:", gold_num)
        print("extracted:", extract_number(text))
        print("––––––––––––––––––\n")

    # now extract the final numeric answer from the full CoT text
    pred_num = extract_number(text)

    correct = (pred_num == gold_num)
    cot_results.append({
        "question": q,
        "gold_num": gold_num,
        "pred_num": pred_num,
        "correct": correct
    })

cot_acc = sum(r["correct"] for r in cot_results) / len(cot_results)
print(f" CoT zero-shot numeric accuracy: {cot_acc:.3%}")


CoT test set:   0%|          | 1/1319 [00:00<17:28,  1.26it/s]

––––––––––––––––––
Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
A: Let’s think step by step. Janet’s ducks lay 16 eggs per day.
She eats three for breakfast every morning and bakes muffins for her friends every day with four.
So, she makes 16 * 4 = <<16*4=48>>48 eggs a day.
Thus, she makes 48 / 3 = <<48/3=12>>12 eggs a day.
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
#### 12
gold_num: 18
extracted: 12
––––––––––––––––––



CoT test set:   0%|          | 2/1319 [00:01<17:21,  1.26it/s]

––––––––––––––––––
Q: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
A: Let’s think step by step.
2*2=<<2*2=4>>4 bolts
4*2=<<4*2=8>>8 bolts
8*2=<<8*2=16>>16 bolts
16*4=<<16*4=64>>64 bolts
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
#### 64
gold_num: 3
extracted: 64
––––––––––––––––––



CoT test set:   0%|          | 3/1319 [00:02<17:19,  1.27it/s]

...

––––––––––––––––––
Q: James decides to run 3 sprints 3 times a week.  He runs 60 meters each sprint.  How many total meters does he run a week?
A: Let’s think step by step.
3 x 60 = <<3*60=180>>180 meters
3 x 60 = <<3*60=180>>180 meters
180 - 180 = <<180-180=60>>60 meters
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week
60 meters/week

gold_num: 540
extracted: 60
––––––––––––––––––



CoT test set:   0%|          | 5/1319 [00:03<17:17,  1.27it/s]

––––––––––––––––––
Q: Every day, Wendi feeds each of her chickens three cups of mixed chicken feed, containing seeds, mealworms and vegetables to help keep them healthy.  She gives the chickens their feed in three separate meals. In the morning, she gives her flock of chickens 15 cups of feed.  In the afternoon, she gives her chickens another 25 cups of feed.  How many cups of feed does she need to give her chickens in the final meal of the day if the size of Wendi's flock is 20 chickens?
A: Let’s think step by step.
The total number of cups of feed given to the chickens is 15*3=<<15*3=45>>45 cups
The total number of cups given to the chickens is 45*3=<<45*3=135>>135 cups
The total number of cups given to the chickens is 135*3=<<135*3=270>>270 cups
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
#### 270
gold_num: 20
extracted: 270
––––––––––––––––––



CoT test set: 100%|██████████| 1319/1319 [17:17<00:00,  1.27it/s]

 CoT zero-shot numeric accuracy: 2.123%


# 9.  Evaluate Using Zero Shot Prompting
We use Zero shot prompting to generate step by step answers and evaluates final numeric accuracy.

In [15]:
import json

# all_results = your list of dicts from the loop above
with open("all_predictions_zeroCOT.json", "w") as f:
    json.dump(cot_results, f, indent=2)

perfect = [r for r in cot_results if r["correct"]]
with open("perfect_subset_zoroCOT.json", "w") as f:
    json.dump(perfect, f, indent=2)

print(f"Saved {len(results)} total preds and {len(perfect)} perfect ones.")


Saved 1319 total preds and 28 perfect ones.
